<a href="https://colab.research.google.com/github/LordLean/Extracting-Green-Bonds-Use-of-Proceeds/blob/main/QA_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retieval

## Answer Retriever


In [10]:
!pip install rank-bm25

!pip install PyPDF2

!pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 214 kB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 4.3 MB/s 


In [11]:
import numpy as np

import tabula
from rank_bm25 import BM25Okapi
from PyPDF2 import PdfReader

In [12]:
class TableReader:

  def __init__(self, pdf):
    self.pdf = pdf
    self.dfs = None

  def read_pages(self, pages="all", multiple_tables=True, stream=True):
    '''
    Return tables discovered within pdf.
    '''
    self.dfs = tabula.read_pdf(self.pdf, pages=pages, multiple_tables=multiple_tables, stream=stream)
    self.__clean_dfs()
    return self.dfs

  def __clean_dfs(self, thresh=2):
    self.dfs = [df.dropna(thresh=thresh) for df in self.dfs]


class Reader:

  def __init__(self, filename):
    self.reader = PdfReader(filename)
    self.tb = TableReader(filename)
    self.page_viewer = {page_num : {} for page_num in range(self.reader.numPages)}
    self.idx2page_item = []
  
  def __extract_text(self,):
    '''
    Page-wise text extraction and tokenize for BM25.
    '''
    text_index_mem = 0
    # List to store each tokenized corpus
    tokenized_corpus_list = []
    for i in range(self.reader.numPages):
      raw_text = self.reader.getPage(i).extractText()
      self.page_viewer[i]["raw_text"] = raw_text
      # Split text
      corpus = raw_text.split("\n \n")
      # Store results.
      self.page_viewer[i]["corpus"] = corpus
      for item in corpus:
        self.idx2page_item.append((i, item)) # page,textItem
      # Tokenize
      tokenized_corpus = [doc.split(" ") for doc in corpus]
      tokenized_corpus_list.append(tokenized_corpus)
    # BM25 computations only after the complete tokenized corpus is collated. 
    # Merge tokenized corpus'.
    tokenized_corpus_complete = [item for sublist in tokenized_corpus_list for item in sublist]
    # BM25
    self.bm25 = BM25Okapi(tokenized_corpus_complete)

  def __extract_tables(self):
    '''
    Page-wise table extractor.
    '''
    for i in range(self.reader.numPages):
      # page=0 will throw error using tabula.
      page = str(i+1)
      self.page_viewer[i]["tables"] = self.tb.read_pages(pages=page)

  def extract_pdf(self):
    # Extract data
    self.__extract_text()
    # self.__extract_tables()

  def print_page(self, page_num):
    '''
    Print separated sections of text given a page.
    '''
    corpus = self.page_viewer[page_num]["corpus"]
    for item in (corpus):
      print("\n{}\n".format("-"*60))
      print(item)
    print("\n{}\n".format("-"*60))
    for df in self.page_viewer[page_num]["tables"]:
      print(df.style)
      display(df)

  def __score(self, queries, weights):
    '''
    Compute the average BM25 score of each given query on each page of text.
    '''
    self.ranked_scores = []
    for query in queries:
      # tokenize query by whitespace.
      tokenized_query = query.split()
      # Compute score.
      doc_scores = self.bm25.get_scores(tokenized_query)
      self.ranked_scores.append(doc_scores)
    # Compute average (weighted) score against all queries.
    if not len(weights):
      # Equal weighting.
      self.average_score = np.average(self.ranked_scores, axis=0)
    elif len(queries) != len(weights):
        # Unequal number of elements.
        raise ValueError("Number of query and weight elements passed must be equal.")
    else:
      # Weighted average.
      self.average_score = np.average(self.ranked_scores, weights=weights, axis=0)
 
  def get_ranked_texts(self, queries, weights=[], n=5):
    '''
    Return n pages which scored highest using BM25.
    '''
    # Run score method to calculate BM25.
    self.__score(queries, weights)
    try:
      idx = sorted(range(len(self.average_score)), key=lambda i: self.average_score[i], reverse=True)[:n]
    except IndexError:
      idx = sorted(range(len(self.average_score)), key=lambda i: self.average_score[i], reverse=True)
    final_results = []
    for i in range(n):
      page_num, text = self.idx2page_item[idx[i]]
      # tables = self.page_viewer[page_num]["tables"]
      # final_results.append({"page_num":page_num, "text":text, "tables":tables})
      final_results.append(text)

    return final_results
    

## Answer Re-ranker (Neural: BERT / T5)

In [13]:
!pip install pygaggle

!pip install transformers==4.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 7.3 MB 9.1 MB/s 
     |████████████████████████████████| 63.3 MB 25 kB/s 
     |████████████████████████████████| 10.6 MB 3.8 MB/s 
     |████████████████████████████████| 2.9 MB 5.8 MB/s 
     |████████████████████████████████| 60 kB 5.7 MB/s 
     |████████████████████████████████| 1.4 MB 44.4 MB/s 
     |████████████████████████████████| 1.1 MB 16.5 MB/s 
     |████████████████████████████████| 86 kB 3.3 MB/s 
     |████████████████████████████████| 1.3 MB 35.5 MB/s 
     |████████████████████████████████| 8.6 MB 21.9 MB/s 
     |████████████████████████████████| 2.2 MB 60.1 MB/s 
     |████████████████████████████████| 184 kB 68.1 MB/s 
     |████████████████████████████████| 3.7 MB 40.3 MB/s 
     |████████████████████████████████| 880 kB 53.4 MB/s 
  Created wheel for pygaggle: filename=p

In [14]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5, MonoBERT

class Reranker:

  def __init__(self):
    self.mono5t = MonoT5()
    self.monobert = MonoBERT()

  def rerank(self, query, texts, method="T5"):
    query = Query(query)
    texts = [Text(text, {"docid" : i}, 0) for i, text in enumerate(texts)]

    if method == "T5":
      reranker = self.mono5t
    if method == "BERT":
      reranker = self.monobert

    reranked = reranker.rerank(query, texts)
    reranked.sort(key=lambda x: x.score, reverse=True)

    return reranked

2022-08-16 07:17:14 [INFO] loader: Loading faiss with AVX2 support.
2022-08-16 07:17:14 [INFO] loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2022-08-16 07:17:14 [INFO] loader: Loading faiss.
2022-08-16 07:17:14 [INFO] loader: Successfully loaded faiss.


# QA Model

In [15]:
# Download zipped model
!gdown 1NBc9MfT4FuchadevFJvcfyBfpIPMILb0

# Unzip
!unzip finbert-pretrain-finetuned-squad.zip

# Delete
!rm finbert-pretrain-finetuned-squad.zip

Downloading...
From: https://drive.google.com/uc?id=1NBc9MfT4FuchadevFJvcfyBfpIPMILb0
To: /content/finbert-pretrain-finetuned-squad.zip
100% 406M/406M [00:07<00:00, 55.1MB/s]
Archive:  finbert-pretrain-finetuned-squad.zip
  inflating: finbert-pretrain-finetuned-squad/config.json  
  inflating: finbert-pretrain-finetuned-squad/pytorch_model.bin  
  inflating: finbert-pretrain-finetuned-squad/special_tokens_map.json  
  inflating: finbert-pretrain-finetuned-squad/tokenizer.json  
  inflating: finbert-pretrain-finetuned-squad/tokenizer_config.json  
  inflating: finbert-pretrain-finetuned-squad/training_args.bin  
  inflating: finbert-pretrain-finetuned-squad/vocab.txt  


In [16]:
# !pip install transformers 

from transformers import pipeline

In [17]:
model_dir = "finbert-pretrain-finetuned-squad"

question_answering = pipeline("question-answering", model=model_dir, tokenizer=model_dir)

In [18]:
context = """
Machine learning (ML) is the study of computer algorithms that improve automatically through experience. It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.
"""
question = "what are the machine learning models based on?"

question, context = "Who was Jim Henson?", "Jim Henson was a nice puppet"

In [19]:
context = """
Vía Célere intends to report on allocation of proceeds on
its website, on an annual basis, until full allocation. The allocation
reporting will include the total amount allocated to projects, the share
of financing vs. refinancing, and unallocated proceeds. In addition,
Vía Célere is committed to reporting on relevant impact metrics, such
as energy consumption reduction (in kWh) or emission reduction (in
tons of CO2e). Sustainalytics views Vía Célere’s allocation and impact
reporting as aligned with market practice.
"""
context2 = """
Blackstone Property Partners Europe Holdings intends
to report on allocation of proceeds on its website on an annual basis
until full allocation or while financing instruments remain
outstanding. In addition, to the extent practicable, the Company
intends to report on relevant impact metrics such as green building
certification level, renewable energy installed capacity and annual
energy savings. Sustainalytics views the allocation and impact
reporting as aligned with market practice.
"""

question = "how often is allocation of proceeds reported?"

In [20]:
result = question_answering(question=question, context=context, device=0)
print("Answer:", result['answer'])
print("Score:", result['score'])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:316: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  fw_args = {k: torch.tensor(v, device=self.device) for (k, v) in fw_args.items()}


Answer: annual basis, until full allocation
Score: 0.20437736809253693


# ICMA Database Upload

In [ ]:
!wget https://www.icmagroup.org/assets/documents/Sustainable-finance/Database/ICMA-Sustainable-Bonds-Database-120822.xlsx

--2022-08-16 07:15:30--  https://www.icmagroup.org/assets/documents/Sustainable-finance/Database/ICMA-Sustainable-Bonds-Database-120822.xlsx
Resolving www.icmagroup.org (www.icmagroup.org)... 91.216.93.249
Connecting to www.icmagroup.org (www.icmagroup.org)|91.216.93.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274575 (268K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘ICMA-Sustainable-Bonds-Database-120822.xlsx’

ICMA-Sustainable-Bo 100%[===================>] 268.14K   273KB/s    in 1.0s    

2022-08-16 07:15:32 (273 KB/s) - ‘ICMA-Sustainable-Bonds-Database-120822.xlsx’ saved [274575/274575]



In [ ]:
import pandas as pd
import openpyxl

In [ ]:
filename = "ICMA-Sustainable-Bonds-Database-120822.xlsx"

# select green bond sheet.
gb_sheet = pd.ExcelFile(filename).sheet_names[0] 

df = pd.read_excel(filename, sheet_name=gb_sheet, header=1)

In [ ]:
# Use openpyxl to load xls with hyperlink text.
wb = openpyxl.load_workbook(filename)
ws = wb[gb_sheet]

hyperlink_list = []

for i in range(len(df)):
  try:
    hyperlink_list.append(ws.cell(row=(3+i), column=6).hyperlink.target)
  except:
    # Nan 
    hyperlink_list.append(None)

# Add list to df.
df["External Review Report Text"] = hyperlink_list

In [ ]:
european = [
    'Spain', "The Netherlands", "Italy", "Sweden", "Norway", "France", "Luxembourg",
    "UK", "Belgium", "Hungary", "Switzerland", "Germany", "Finland", "Iceland", "Poland",
    "Czech Republic", "Denmark", "Ireland", "Greece", "Guernsey", "Austria", "Latvia",
    "Lithuania", "Romania", "Slovenia", "Slovakia",
]
sector = "Corporate-Real Estate"
external = "SUSTAINALYTICS" # second-party opinion


df = df.loc[
    (df["Jurisdiction"].isin(european)) &
    (df["Issuer Category/Sector"] == sector) &
    (df["External Review Report"] == external)
] 

files = df["External Review Report Text"].to_list()

In [ ]:
files

['https://www.icmagroup.org/Emails/icma-vcards/Blackstone_External%20Review%20Report.pdf',
 'http://www.icmagroup.org/Emails/icma-vcards/Castellum_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/Cibus_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/CTP_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/Deutsche%20Wohnen_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/Globalworth_External%20Review%20Report.pdf',
 'http://www.icmagroup.org/Emails/icma-vcards/ICADE_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/Johnson_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/Specialfastigheter-External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/tritax-eurobox_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/Vesteda_External%20Review%20Report.pdf',
 'htt

In [ ]:
name2url = {link.strip().rsplit('/', 1)[-1] : link.strip() for link in files}
url2name = {link.strip() : link.strip().rsplit('/', 1)[-1]for link in files}
url2name

{'https://www.icmagroup.org/Emails/icma-vcards/Blackstone_External%20Review%20Report.pdf': 'Blackstone_External%20Review%20Report.pdf',
 'http://www.icmagroup.org/Emails/icma-vcards/Castellum_External%20Review%20Report.pdf': 'Castellum_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/Cibus_External%20Review%20Report.pdf': 'Cibus_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/CTP_External%20Review%20Report.pdf': 'CTP_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/Deutsche%20Wohnen_External%20Review%20Report.pdf': 'Deutsche%20Wohnen_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/Globalworth_External%20Review%20Report.pdf': 'Globalworth_External%20Review%20Report.pdf',
 'http://www.icmagroup.org/Emails/icma-vcards/ICADE_External%20Review%20Report.pdf': 'ICADE_External%20Review%20Report.pdf',
 'https://www.icmagroup.org/Emails/icma-vcards/Johnson_External%20Revi

In [ ]:
# Create documents folder
!mkdir documents

In [ ]:
import os.path
import urllib.request

for link, name in url2name.items():
    filename = os.path.join('./documents', name)
    if not os.path.isfile(filename):
        print('Downloading: ' + filename)
        try:
            urllib.request.urlretrieve(link, filename)
        except Exception as inst:
            print(inst)
            print('  Encountered unknown error. Continuing.')

Downloading: ./documents/Blackstone_External%20Review%20Report.pdf
Downloading: ./documents/Castellum_External%20Review%20Report.pdf
Downloading: ./documents/Cibus_External%20Review%20Report.pdf
Downloading: ./documents/CTP_External%20Review%20Report.pdf
Downloading: ./documents/Deutsche%20Wohnen_External%20Review%20Report.pdf
Downloading: ./documents/Globalworth_External%20Review%20Report.pdf
Downloading: ./documents/ICADE_External%20Review%20Report.pdf
Downloading: ./documents/Johnson_External%20Review%20Report.pdf
Downloading: ./documents/Specialfastigheter-External%20Review%20Report.pdf
Downloading: ./documents/tritax-eurobox_External%20Review%20Report.pdf
Downloading: ./documents/Vesteda_External%20Review%20Report.pdf
Downloading: ./documents/V%C3%ADa%20C%C3%A9lere_External%20Review%20Report.pdf
Downloading: ./documents/Vonovia_External%20Review%20Report.pdf
Downloading: ./documents/Xior_External%20Review%20Report.pdf


# Run Questions

In [21]:
from tqdm.notebook import tqdm_notebook

In [22]:
# Set of query, question pairs per area of interest.

# For each area of questioning: list of IR queries with paired question.
query_questioner = {
    # Area of investigation.
    "Alignment" : [
        # (list of IR queries, question_for_QA_model)
        (["four core components of the GBP", "Alignment with Green Bond Principles"], "sustainalytics is of the opinion that the Bonds are what?"),
        (["Green finance framework"], "the green finance framework is what?"),
    ],
    "SDG Goals" : [
        (["UN Sustainable Development Goals", "SDG"], "Which sustainable development goals are advanced?"),
    ],
    "Use of Proceeds" : [
        ([" eligible category for the use of proceeds"], "What are the eligible categories for the use of proceeds?"),
        (["eligible category for the use of proceeds", "UN Sustainable Development Goals", "SDG"], "What do the eligible categories lead to?"),
    ],
    "Project Evaluation" : [
        (["Project Evaluation Selection"], "who manages evaluating and selecting projects?"),
    ],
    "Management of Proceeds" : [
        (["management proceeds"], "who is responsible for management of proceeds?"),
        (["unallocated proceeds"], "unallocated proceeds will be held where?"),
    ],
    "Reporting" : [
        (["reporting", "impact reporting"], "what is reported?"),
        (["until full allocation", "report", "on the allocation of the net proceeds of issued green finance instruments"], "on what basis?"),
    ],
}

In [23]:
# dataframe to hold results
df_test = pd.DataFrame(columns=["Company", "Area of Interest", "IR Query", "Question", "Score", "Answer", "Answer Sentence", "Answer Full Text", "Priority Flag"])

In [24]:
class QueryAnswer:

  def __init__(self, company, area, retrieval_query, qa_question, score, answer, answer_sentence, answer_full_text):
    self.company = company
    self.area = area
    self.retrieval_query = retrieval_query
    self.qa_question = qa_question
    self.score = score
    self.answer = answer
    self.answer_sentence = answer_sentence
    self.answer_full_text = answer_full_text

In [25]:
reranker = Reranker()

In [26]:
bm25_count = 30
reranked_count = 15

In [27]:
for company in tqdm_notebook(name2url.keys()):
  # Extract PDF
  print(company)
  filename = "documents/{}".format(company)
  reader = Reader(filename)
  reader.extract_pdf()
  # Iterate through query, question pairs 
  for area, pair_list in tqdm_notebook(query_questioner.items()):
    for queries, question in pair_list:

      # Get BM25 rankings
      texts = reader.get_ranked_texts(queries, n=bm25_count)
      # Rerank
      reranked = reranker.rerank(queries, texts, method="T5")
      reranked = [item for item in reranked if len(item.text.strip())>0]
      reranked = reranked[:reranked_count]

      # List to store results.
      queryAnswer_list = []
      # Iterate through reranker list of text.
      for i in range(len(reranked)):
        # Feed context and question in QA model, return top 3 results.
        context = reranked[i].text
        results = question_answering(question=question, context=context, device=0, topk=3)
        # Iterate through each result to append to results list. -- ultimately we want to consider all results, including those on repeated texts, if those results best answered the question.
        for result in results:
          # Match answer to sentence
          all_stops = [i for i, ltr in enumerate(context) if ltr == "."]
          sentence = ""
          try:
            if all_stops:
              if result["end"] <= all_stops[0]:
                sentence = context[:all_stops[0]]
              elif result["start"] >= all_stops[-1]:
                sentence = context[all_stops[-1]:]
              else:
                for i, stop_idx in enumerate(all_stops):
                  if result["start"] >= stop_idx:
                    try:
                      sentence = context[all_stops[i] : all_stops[i+1]]
                    except IndexError:
                      sentence = context[all_stops[i]:]
            else:
              sentence = context
          except:
            pass
          # Save discovered results in QueryAnswer obj.
          item = QueryAnswer(
              company = company,
              area = area,
              retrieval_query = queries,
              qa_question = question,
              score = result["score"],
              answer = result["answer"],
              answer_sentence = sentence,
              answer_full_text=context
          )
          queryAnswer_list.append(item)

      # Get top 3 results based on QA returned score.
      top_n = 3
      queryAnswer_list.sort(key = lambda x : x.score, reverse=True)
      final_queryAnswers = queryAnswer_list[:top_n]
      # Add new record to dataframe.
      for i, result in enumerate(final_queryAnswers):
        new_row = {
            "Company" : result.company,
            "Area of Interest" : result.area,
            "IR Query" : result.retrieval_query,
            "Question" : result.qa_question,
            "Score" : result.score,
            "Answer" : result.answer,
            "Answer Sentence" : result.answer_sentence,
            "Answer Full Text" : result.answer_full_text,
            "Priority Flag" : i
        }
        df_test = df_test.append(new_row, ignore_index=True)

Blackstone_External%20Review%20Report.pdf


2022-08-16 07:20:13 [WARNING] _utils:  impossible to decode XFormObject /Meta29
2022-08-16 07:20:13 [WARNING] _utils:  impossible to decode XFormObject /Meta30
2022-08-16 07:20:13 [WARNING] _utils:  impossible to decode XFormObject /Meta31
2022-08-16 07:20:13 [WARNING] _utils:  impossible to decode XFormObject /Meta32


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:173: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."



Castellum_External%20Review%20Report.pdf


2022-08-16 07:39:11 [WARNING] _utils:  impossible to decode XFormObject /Meta31
2022-08-16 07:39:11 [WARNING] _utils:  impossible to decode XFormObject /Meta32
2022-08-16 07:39:11 [WARNING] _utils:  impossible to decode XFormObject /Meta33
2022-08-16 07:39:11 [WARNING] _utils:  impossible to decode XFormObject /Meta34


...

2022-08-16 09:30:12 [WARNING] _utils:  impossible to decode XFormObject /Meta28
2022-08-16 09:30:12 [WARNING] _utils:  impossible to decode XFormObject /Meta29
2022-08-16 09:30:12 [WARNING] _utils:  impossible to decode XFormObject /Meta30
2022-08-16 09:30:12 [WARNING] _utils:  impossible to decode XFormObject /Meta31



Vonovia_External%20Review%20Report.pdf


2022-08-16 09:43:12 [WARNING] _utils:  impossible to decode XFormObject /Meta30
2022-08-16 09:43:12 [WARNING] _utils:  impossible to decode XFormObject /Meta31
2022-08-16 09:43:12 [WARNING] _utils:  impossible to decode XFormObject /Meta32
2022-08-16 09:43:12 [WARNING] _utils:  impossible to decode XFormObject /Meta33



Xior_External%20Review%20Report.pdf


2022-08-16 09:43:12 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:12 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  impossible to decode XFormObject /Meta35
2022-08-16 09:43:13 [WARNING] _utils:  i

In [31]:
df_test.loc[
    (df_test["Area of Interest"] == "Use of Proceeds") &
    (df_test["Priority Flag"] == 0) &
    (df_test["Question"] == "What are the eligible categories for the use of proceeds?")
] 

,Company,Area of Interest,IR Query,Question,Score,Answer,Answer Sentence,Answer Full Text,Priority Flag
9,Blackstone_External%20Review%20Report.pdf,Use of Proceeds,[ eligible category for the use of proceeds],What are the eligible categories for the use o...,0.770669,"Green Buildings, Renewable Energy, Energy Effi...",12 \nThe eligible categories for the use of p...,12 \nThe eligible categories for the use of p...,0
39,Castellum_External%20Review%20Report.pdf,Use of Proceeds,[ eligible category for the use of proceeds],What are the eligible categories for the use o...,0.735204,Project Evaluation and Selection,. All pr ojects financed by Green \nBonds pro...,2.3 Castellum’s Green Bond \nCastellum intend...,0
69,Cibus_External%20Review%20Report.pdf,Use of Proceeds,[ eligible category for the use of proceeds],What are the eligible categories for the use o...,0.755215,Green and Energy Efficient Buildings,. This assessment is based on \nthe following:...,Evaluation Summary \nSustainalytics is of the...,0
99,CTP_External%20Review%20Report.pdf,Use of Proceeds,[ eligible category for the use of proceeds],What are the eligible categories for the use o...,0.833602,Green Buildings and Renewable Energy,9 The eligible categories for the use of proc...,9 The eligible categories for the use of proc...,0
129,Deutsche%20Wohnen_External%20Review%20Report.pdf,Use of Proceeds,[ eligible category for the use of proceeds],What are the eligible categories for the use o...,0.840007,credible and impactful,.3 \nScope of work and limitations of Sustain...,2 Introduction \nDeutsche Wohnen SE Group (...,0
159,Globalworth_External%20Review%20Report.pdf,Use of Proceeds,[ eligible category for the use of proceeds],What are the eligible categories for the use o...,0.806361,credible and impactful,.2 \nScope of work and limitations of Sustain...,2 Introduction \nGlobalworth Real Estate Inve...,0
189,ICADE_External%20Review%20Report.pdf,Use of Proceeds,[ eligible category for the use of proceeds],What are the eligible categories for the use o...,0.533009,Icade’s Green Bond,". Furthermore, to be eligible for the use of p...","However, Sustainalytics notes that t he majori...",0
219,Johnson_External%20Review%20Report.pdf,Use of Proceeds,[ eligible category for the use of proceeds],What are the eligible categories for the use o...,0.904923,credible and impactful,.3 \nScope of work and limitations of Sustain...,2 Introduction \nJohnson Controls Internation...,0
249,Specialfastigheter-External%20Review%20Report.pdf,Use of Proceeds,[ eligible category for the use of proceeds],What are the eligible categories for the use o...,0.260062,Renewable energy \n ☒ Energy efficiency,Use of proceeds categories as per GBP: \n☒ Re...,Use of proceeds categories as per GBP: \n☒ Re...,0
279,tritax-eurobox_External%20Review%20Report.pdf,Use of Proceeds,[ eligible category for the use of proceeds],What are the eligible categories for the use o...,0.862161,"Green Buildings, Renewable Energy, and Energy ...",11 The eligible categories for the use of proc...,11 The eligible categories for the use of proc...,0


In [30]:
from google.colab import files
df_test.to_csv("experimental_results.csv")
files.download("experimental_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>